In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk import download
import string
import re
from gensim.models import Word2Vec
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from fuzzywuzzy import fuzz

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.116.253.138:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.116.253.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.116.253.138:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_json('./drive/My Drive/edustack/news.json', orient = 'records', lines = True)

In [ ]:
df.columns

Index(['category', 'headline', 'authors', 'link', 'short_description', 'date'], dtype='object')

In [ ]:
'''Some contractions, rules and token replace dictionaries which will help in text preprocessing'''
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"its":"it is",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

rules = {
    "'t": " not",
    "'cause": " because",
    "'ve": " have",
    "'t": " not",
    "'s": " is",
    "'d": " had"
}

punctuations = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", 
                '$', '&', '/', '[', ']','>', '%', '=', '#', '*', '+', '\\', '•', '~', 
                '@', '£', '·', '_', '{', '}', '©', '^','®', '`', '<', '→', '°', '€', '™', 
                '›', '♥', '←', '×', '§', '″', '′', 'Â', '█','½', 'à', '…', '“', '★', '”', 
                '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶','↑', '±', '¿', '▾', '═', 
                '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼','⊕', '▼', '▪', '†',
                '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲','è', '¸', '¾', 
                'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪','╩', 
                '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 
                'ï', 'Ø', '¹', '≤', '‡', '√'] + list(string.punctuation)
token_replace = {
    'usepackage':'latex',
    'orf19':'gene',
    'documentclass':'latex',
    'magento':'open-source e-commerce',
    'appium':'web-app',
    'tikz':'programming language',
    'tikzpicture':'programming language',
    'openvpn':'vpn',
    'httpclient':'http client',
    'arraylist':'array list',
    'jsonobject': 'json',
    'artifactid':'xml',
    'hwnd':'os'
}
punctuations_in_embeddings = {',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', 
                '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '{', '}', '©', '^', '®', 
                '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
                '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '═', 
                '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', 
                '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
                '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔',
                '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '!', '"', '#', '$', '%', '&', "'", '(', ')', 
                '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '{', 
                '|', '}', '~'}

In [ ]:
'''Pipeline for text preprocessing. Check coverage computes what %age of tokens in the text data 
are covered by Embeddings.
preprocess_text is used to clean text.
'''
def improve_text(dframes):
    def preprocess_text(s):
        s = s.lower()
        '''Expanding contractions...'''
        for key, value in contractions.items():
            s = s.replace(key, f' {value} ')
        for key, value in rules.items():
            s = s.replace(key, f' {value} ')
        '''Fixing punctuations...'''
        for punct in punctuations:
            if punct in punctuations_in_embeddings:
                s = s.replace(punct, f' {punct} ')
            else:
                s = s.replace(punct, ' ')
        '''Replacing few tokens with its similar word/group of words...'''
        for key, value in token_replace.items():
            s = s.replace(key, value)
        '''Removing HTML tags'''
        s = re.sub('<.*?>', ' ', s)
        s = re.sub('\s+', ' ', s)
        return s
    print("Applying Preprocessing.....")
    for each_df in dframes:
        each_df['clean_headline'] = each_df['headline'].apply(preprocess_text)
        each_df['clean_desc'] = each_df['short_description'].apply(preprocess_text)
    print("Preprocessing Finished")

In [ ]:
improve_text([df])

Applying Preprocessing.....
Preprocessing Finished


In [ ]:
!pip install fuzzywuzzy

In [ ]:
def fetch_words(s, use_stopwords = False, ignore_stopwords = False):
    if ignore_stopwords == True:
        return {word for word in s.split() if word not in stop_words}
    elif use_stopwords:
        return {word for word in s.split() if word in stop_words}
    else:
        return set(s.split())
def text_features(row, use_stopwords, is_token, return_max):
    t1, t2 = None, None
    if use_stopwords == True:
        t1 = fetch_words(row[0], use_stopwords = True)
        t2 = fetch_words(row[1], use_stopwords = True)
    elif is_token == True:
        t1 = fetch_words(row[0], ignore_stopwords = True)
        t2 = fetch_words(row[1], ignore_stopwords = True)
    else:
        t1 = fetch_words(row[0])
        t2 = fetch_words(row[1])
    if return_max:
        try:
            ans = len(t1.intersection(t2))/max(len(t1), len(t2))
        except:
            return 0
    else:
        try:
            ans = len(t1.intersection(t2))/min(len(t1), len(t2))
        except:
            return 0
    return ans

def get_ratio(row, is_token = False):
    if is_token == True:
        t1 = len(set([word for word in row[0].split() if word not in stop_words]))
        t2 = len(set([word for word in row[1].split() if word not in stop_words]))
    t1 = len(set([word for word in row[0].split()]))
    t2 = len(set([word for word in row[1].split()]))
    try:
        return t1/t2
    except:
        return 0
def get_fuzzy_ratio(row, ratio, partial_ratio, token_sort_ratio, token_set_ratio):
    if ratio:
        return fuzz.ratio(row[0], row[1])/100
    elif partial_ratio:
        return fuzz.partial_ratio(row[0], row[1])/100
    elif token_sort_ratio:
        return fuzz.token_sort_ratio(row[0], row[1])/100
    else:
        return fuzz.token_set_ratio(row[0], row[1])/100

In [ ]:
df.shape

(2009, 22)

In [ ]:
# temp_df = df.copy()
#df = df.sample(frac = 0.01)
df = temp_df.copy()

In [ ]:
feature_engg_cols = ['cwc_min_headline_desc',
    'cwc_max_headline_desc', 'csc_min_headline_desc',
    'csc_max_headline_desc', 'ctc_min_headline_desc',
    'ctc_max_headline_desc', 
    'word_headline_desc_ratio', 'token_headline_desc_ratio',
    'ratio_headline_desc', 'partial_ratio_headline_desc',
    'token_sort_ratio_headline_desc',
    'token_set_ratio_headline_desc']

df['cwc_min_headline_desc'] = df[['clean_headline', 'clean_desc']].apply(func = text_features, 
                                                    axis = 1, args = (False, False, False))
df['cwc_max_headline_desc'] = df[['clean_headline', 'clean_desc']].apply(func = text_features, 
                                                    axis = 1, args = (False, False, True))
df['csc_min_headline_desc'] = df[['clean_headline', 'clean_desc']].apply(func = text_features, 
                                                    axis = 1, args = (True, False, False))
df['csc_max_headline_desc'] = df[['clean_headline', 'clean_desc']].apply(func = text_features, 
                                                    axis = 1, args = (True, False, True))
df['ctc_min_headline_desc'] = df[['clean_headline', 'clean_desc']].apply(func = text_features, 
                                                    axis = 1, args = (False, True, False))
df['ctc_max_headline_desc'] = df[['clean_headline', 'clean_desc']].apply(func = text_features, 
                                                    axis = 1, args = (False, True, True))

df['word_headline_desc_ratio'] = df[['clean_headline', 'clean_desc']].apply(
        func = get_ratio, axis = 1, args = (False, ))
df['token_headline_desc_ratio'] = df[['clean_headline', 'clean_desc']].apply(
        func = get_ratio, axis = 1, args = (True, ))
df['ratio_headline_desc'] = \
df[['clean_headline', 'clean_desc']].apply(func = get_fuzzy_ratio, 
                                                    axis = 1, args = (True, False, False, False))
df['partial_ratio_headline_desc'] = \
df[['clean_headline', 'clean_desc']].apply(func = get_fuzzy_ratio, 
                                                    axis = 1, args = (False, True, False, False))
df['token_sort_ratio_headline_desc'] = \
df[['clean_headline', 'clean_desc']].apply(func = get_fuzzy_ratio, 
                                                    axis = 1, args = (False, False, True, False))
df['token_set_ratio_headline_desc'] = \
df[['clean_headline', 'clean_desc']].apply(func = get_fuzzy_ratio, 
                                                    axis = 1, args = (False, False, False, True))


In [ ]:
# df.to_csv('./drive/My Drive/edustack/data1.csv', index = False)
fe = df[feature_engg_cols].values.astype("float32")

In [ ]:
df['combine_text'] = df.clean_headline + ' ' + df.clean_desc 
all_texts = df['combine_text'].tolist() 
all_texts = [text.split() for text in all_texts]
model = Word2Vec(size = 300, min_count = 1, window = 5)
model.build_vocab(all_texts)
model.intersect_word2vec_format('./drive/My Drive/google_qa/fasttext-wiki-news-subwords-300.gz', lockf = 1)
model.train(all_texts, total_examples = model.corpus_count, epochs = 5)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(26761715, 34919320)

In [ ]:
def category_cleaner(x):
    if x == 'THE WORLDPOST':
        return 'WORLDPOST'
    elif x == 'PARENTING':
        return 'PARENTS'
    elif x == 'ARTS' or x == 'CULTURE & ARTS':
        return 'ARTS & CULTURE'
    elif x == 'STYLE':
        return 'STYLE & BEAUTY'
    elif x == 'COLLEGE':
        return 'EDUCATION'
    elif x == 'TASTE':
        return 'FOOD & DRINK'
    else:
        return x
    
df['category'] = df.category.apply(category_cleaner)

In [ ]:
counter = Counter()
for text in all_texts:
  counter.update(text)
vocab = {}
vocab['token2id'] = {key:id+1 for id, (key, _) in enumerate(counter.items())}
vocab['id2token'] = {value:key for key, value in vocab['token2id'].items()}
vocab['word_freq'] = dict(counter)

In [ ]:
def build_embedding_matrix(vocab, texts, embedd_size, model):
  n = len(vocab['token2id'])+1
  embedding_matrix = np.zeros((n, embedd_size))
  for text in texts:
    for key in text:
      word = key
      try:
        embedding_matrix[vocab['token2id'][word]] = model.wv[word]
        continue
      except:
        pass
  return embedding_matrix

In [ ]:
def get_token_ids(texts, max_length):
  tokens = []
  for text in texts:
    tmp_tokens = []
    if len(text.split()) > max_length:
      for each in (text.split()[:(max_length//2)] + text.split()[-(max_length//2):]):
        tmp_tokens.append(vocab['token2id'].get(each, 0))
      tokens.append(tmp_tokens)
    else:
      for each in (text.split()[:max_length]):
        tmp_tokens.append(vocab['token2id'].get(each, 0))
      tokens.append(tmp_tokens)
  return tf.keras.preprocessing.sequence.pad_sequences(tokens, padding="post", maxlen=max_length)

In [ ]:
embedding_matrix = build_embedding_matrix(vocab, all_texts, 300, model)

In [ ]:
#df['clean_combined'] = df['clean_headline'] + ' ' + df['clean_desc']
df['clean_headline'] = df.clean_headline.apply(lambda x: x.strip())
df['clean_desc'] = df.clean_desc.apply(lambda x: x.strip())

In [ ]:
MAX_LENGTH = 60
data = {}
data['headline'] = get_token_ids(df['clean_headline'], MAX_LENGTH)
data['desc'] = get_token_ids(df['clean_desc'], MAX_LENGTH)
targets = df.category.astype('category').cat.codes.values
targets = tf.keras.utils.to_categorical(targets, num_classes = 34)

In [ ]:
class Accuracy(tf.keras.metrics.Metric):
    def __init__(self, name = 'accuracy', batch_size = 32, **kwrgs):
        super(Accuracy, self).__init__(name = name, **kwrgs)
        self.correct_predictions = self.add_weight(name = 'one',initializer = 'zeros')
        self.total_predictions = self.add_weight(name = 'two', initializer = 'zeros')
        self.batch_size = batch_size
    def update_state(self, y_true, y_pred, sample_weight = None):
        predictions = tf.argmax(y_pred, axis = 1)
        true_predictions = tf.argmax(y_true, axis = 1)
        correct_values = (tf.cast(true_predictions, "int32") == tf.cast(predictions, "int32"))
        correct_values = tf.cast(correct_values, "float32")
        self.correct_predictions.assign_add(tf.reduce_sum(correct_values))
    def result(self):
        self.total_predictions.assign_add(tf.constant(self.batch_size, dtype = "float32"))
        return self.correct_predictions/self.total_predictions
    def reset_state(self):
        self.correct_predictions.assign(0)
        self.total_predictions.assign(0)

In [ ]:
def get_generator(X, batch_size = 32, training = True):
    Y = X[1]
    N = X[0][0].shape[0]
    indexes = np.arange(N)
    if training == True:
        indexes = np.random.permutation(indexes)
    def generator():
        for i in indexes:
            yield {"input_1": X[0][0][i], "input_2":X[0][1][i], "input_3":X[0][2][i]}, Y[i]
    return tf.data.Dataset.from_generator(generator, 
                    output_types = ({"input_1": tf.int32, "input_2":tf.int32, "input_3":tf.float32}, tf.float32)).repeat().batch(batch_size)

In [ ]:
def c_lstm(max_length):
  i1 = tf.keras.layers.Input(shape = (max_length), name = 'input_1')
  i2 = tf.keras.layers.Input(shape = (max_length), name = 'input_2')
  fe_ = tf.keras.layers.Input(shape = (fe.shape[1]), name = 'input_3')
  e1 = tf.keras.layers.Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], weights = [embedding_matrix], trainable = False)(i1)
  e2 = tf.keras.layers.Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], weights = [embedding_matrix], trainable = False)(i2)
  
  
  lstm_head = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(e1)
  lstm_head_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=False))(tf.keras.layers.SpatialDropout1D(rate = 0.2)(lstm_head))
  
  lstm_desc = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(e2)
  lstm_desc_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=False))(tf.keras.layers.SpatialDropout1D(rate = 0.2)(lstm_desc))
  


#   avg_head = tf.keras.layers.GlobalAveragePooling1D()(lstm_head)
#   avg_desc = tf.keras.layers.GlobalAveragePooling1D()(lstm_desc)
  
  #concat = tf.keras.layers.Concatenate()([lstm_head, lstm_desc_2)
  
  #drop = tf.keras.layers.Dropout(rate = 0.2)(concat)
  
  d1 = tf.keras.layers.Dense(512, activation = 'relu')(tf.keras.layers.Dropout(rate = 0.2)(lstm_head_2))
  d2 = tf.keras.layers.Dense(512, activation = 'relu')(tf.keras.layers.Dropout(rate = 0.2)(lstm_desc_2))
  concat = tf.keras.layers.Concatenate()([d1, d2, fe_])
  out = tf.keras.layers.Dense(34, activation = 'softmax')(concat)

  model = tf.keras.Model(inputs = [i1, i2, fe_], outputs = [out])

  return model

In [ ]:
kf = StratifiedKFold(n_splits = 5).split(X = df.link, y = df.category)
lr_sched = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3)#*(0.7 ** ((epoch%4))))
for fold, (train_idx, valid_idx) in enumerate(kf):
    if fold != 1:
        continue
    tf.keras.backend.clear_session()
    BATCH_SIZE = 128
    train_inputs = ((data['headline'][train_idx],data['desc'][train_idx], fe[train_idx]), targets[train_idx]) 
    valid_inputs = ((data['headline'][valid_idx],data['desc'][valid_idx], fe[valid_idx]), targets[valid_idx])
    train_dataset = get_generator(train_inputs, BATCH_SIZE)
    valid_dataset = get_generator(valid_inputs, BATCH_SIZE, False)
    with tpu_strategy.scope():
        model = c_lstm(MAX_LENGTH)
        model.compile(tf.keras.optimizers.Adam(), 
                    loss = tf.keras.losses.CategoricalCrossentropy(),
                    metrics = ['accuracy'])
                    #metrics = [Accuracy(batch_size = BATCH_SIZE)])
    # model.fit(train_dataset, steps_per_epoch = train_idx.shape[0]//BATCH_SIZE, validation_data = valid_dataset,
    #           validation_steps = valid_idx.shape[0]//BATCH_SIZE, epochs = 10)
    model.fit(x = train_inputs[0], y = train_inputs[1], validation_data = valid_inputs, batch_size = BATCH_SIZE, epochs = 5)

Epoch 1/5
   2/1256 [..............................] - ETA: 53:14 - loss: 3.4760 - accuracy: 0.0938  WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0029s vs `on_train_batch_end` time: 0.0515s). Check your callbacks.


1256/1256 [==============================] - ETA: 0s - loss: 1.5124 - accuracy: 0.5786WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_test_batch_end` time: 0.0231s). Check your callbacks.


1256/1256 [==============================] - 88s 70ms/step - loss: 1.5124 - accuracy: 0.5786 - val_loss: 1.3514 - val_accuracy: 0.6102
Epoch 2/5
1256/1256 [==============================] - 76s 60ms/step - loss: 1.1874 - accuracy: 0.6561 - val_loss: 1.2193 - val_accuracy: 0.6500
Epoch 3/5
1256/1256 [==============================] - 76s 60ms/step - loss: 1.0344 - accuracy: 0.6940 - val_loss: 1.1915 - val_accuracy: 0.6587
Epoch 4/5
1256/1256 [==============================] - 75s 60ms/step - loss: 0.9010 - accuracy: 0.7260 - val_loss: 1.2004 - val_accuracy: 0.6535
Epoch 5/5
1256/1256 [==============================] - 76s 60ms/step - loss: 0.7694 - accuracy: 0.7606 - val_loss: 1.2388 - val_accuracy: 0.6464
